<a href="https://colab.research.google.com/github/LanaSina/chainer_prednet/blob/benham_speed/chainer_prednet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install tb chainer

Mount drive (fpsi images are in .zip file inside drive)


In [0]:
# check python version (this should be python 2)
# change python version through menu: Runtime -> Change runtime type
# choose python 2 and gpu
import sys
print(sys.version)


In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#%pip install tensorboard-chainer
%pip install -U PyYAML

Pull prednet files from github


In [0]:
 !rm -r chainer_prednet
 #!rm -r images
  
 !git clone --branch benham_speed https://github.com/LanaSina/chainer_prednet.git --recurse-submodules


Import images

In [0]:
# !unzip -q 'drive/My Drive/AI/prednet_private/expanding.zip' -d 'expanding'

!rm -r 'imported/'
!mkdir "imported"
!cp -r '/content/drive/My Drive/AI/prednet_private/test/input_images/' 'imported/'
#!mv "benham/Benham_test_data_2/" "benham/input_images/"

In [0]:
!rm -r "imported/input_images/.DS_Store"
!rm -r "imported/input_images/.ipynb_checkpoints"


In [0]:
#build text files
#!rm -r "imported/input_images/.DS_Store"
%run 'chainer_prednet/generate_imagelist.py' 'imported/' '1' -tr 10
#'drive/My Drive/AI/prednet_private/lines_forward_5'

In [0]:
!cp '/content/drive/My Drive/AI/prednet_private/fpsi_500000_20v.model' '.'

In [0]:

# !ls '/content/drive/My Drive/AI/prednet_private/'
!cp '/content/drive/My Drive/AI/prednet_private/mit_500000_Ax4_RGB.model' '.'



FLOW

In [0]:
# test optical flow
%run 'chainer_prednet/optical_flow_tests.py' -i 'imported/' -m 'fpsi_500000_20v.model' -o 'flow_test' -r '10'


In [0]:
# mirrroring
%run "chainer_prednet/utilities/mirror_images.py" -i "imported/input_images" -o "mirrored"

In [0]:
!rm -r "result"
# %run 'chainer_prednet/PredNet/main.py' --images 'imported' --initmodel '1000000_Ax4_RGB.model' --input_len 10 --test
%run 'chainer_prednet/PredNet/main.py' --images 'mirrored' --initmodel 'fpsi_500000_20v.model' --input_len 10 --test --g 0 --ext_t -1
#--ext 3 --ext_t 4


In [0]:
!zip -r "result_images.zip" "flow_test" 
from google.colab import files
files.download('result_images.zip') 


In [0]:
run "chainer_prednet/batch_rename_results.py" "result/"

In [0]:
run "chainer_prednet/verify_predicted_colors.py" "result/" "fpsi/input_images/"  "greyscale_input/input_images" "greyscale_analysis"


In [0]:
# !apt install ffmpeg

#!ffmpeg -framerate 10 -pattern_type glob -i "greyscale_analysis/col_diff/*.png" 'greyscale_analysis/col_diff/_out.mp4'
#!ffmpeg -framerate 10 -pattern_type glob -i "greyscale_analysis/col_match/*.png" 'greyscale_analysis/col_match/_out.mp4'
#!ffmpeg -framerate 10 -pattern_type glob -i "greyscale_analysis/col_overlay/*.png" 'greyscale_analysis/col_overlay/_out.mp4'
!ffmpeg -framerate 10 -pattern_type glob -i "result/*.png" 'result/_out.mp4'

In [0]:
#files.download('greyscale_analysis/col_diff/_out.mp4') 
#files.download('greyscale_analysis/col_match/_out.mp4') 
#files.download('greyscale_analysis/col_overlay/_out.mp4') 
#files.download('result/_out.mp4')
%run "optical_flow/flow_run.py" 


In [0]:
# image analysis
%run "chainer_prednet/images_analysis.py" -i "fpsi/input_images"

# correct 0 padding
run "chainer_prednet/batch_padding.py" "fpsi/input_images" --p '8' --prefix 'frame_'

# create black and white input
!rm -r "greyscale_input"
%run 'chainer_prednet/convert_to_greyscale.py' 'fpsi/input_images' '100' --s '15000'